In [ ]:
# Imports 

import networkx as nx
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
COUNT = 'count'

# Parameters
growth_rate = 0.5
death_rate = 0.00
expulsion_rate = 0.0

initial_load = 100
initial_infected_node = 0

time_limit = 10

split_method = 'horsfield'

In [ ]:
# Network
bronchial_tree = nx.Graph()

# Trachea
bronchial_tree.add_edge(0,1)
origin = 0

# MAIN (PRIMARY) BRONCHI
bronchial_tree.add_edges_from([(1,2), (2,3), (1,4)])

# LOBAR BRONCHI
bronchial_tree.add_edges_from([(2,5), (5,6), (3,7), (3,8), (8,9), (9,10), (10,11), (4,12), (12,13), (12,14), (4,15), 
                               (15,16), (16,17)])

# SEGMENTAL BRONCHI
bronchial_tree.add_edges_from([(5,18), (6,19), (6,20), 
                               (7,21), (7,22), 
                               (8,23), (9,24), (10,25), (11,26),(11,27), 
                               (13,28), (13,29), 
                               (14,30), (14,31), 
                               (15,32), (16,33), (17,34),(17,35)])

# Bronchopulmonary_segments
terminal_nodes = [n for n in bronchial_tree.nodes() if bronchial_tree.degree(n) == 1 and n != origin]
non_terminal_nodes = [n for n in bronchial_tree.nodes() if bronchial_tree.degree(n) != 1]

# Ordered list of nodes (top to bottom)
ordered_list = [origin]
counter = 0
while len(ordered_list) != len(bronchial_tree.nodes()):
    node = ordered_list[counter]
    neighbours = bronchial_tree.neighbors(node)
    ordered_list += [n for n in neighbours if n not in ordered_list]
    counter += 1

# Nodes in Lobes
lobe_nodes = dict()
lobe_nodes['right_superior'] = range(18,21)
lobe_nodes['right_middle'] = range(21,23)
lobe_nodes['right_inferior'] = range(23,28)
lobe_nodes['left_superior'] = range(28,32)
lobe_nodes['left_inferior'] = range(32,36)

# Nodes in Lungs
lung_nodes = dict()
lung_nodes['right'] = lobe_nodes['right_superior'] + lobe_nodes['right_middle'] + lobe_nodes['right_inferior']
lung_nodes['left'] = lobe_nodes['left_superior'] + lobe_nodes['left_inferior']

In [ ]:
# POSITIONING NODES
# Arbitrary - purely for convenience

pos = dict()
pos[0] = (5,10)

pos[1] = (5,8)
pos[2] = (4,7)
pos[3] = (3.5,5)
pos[4] = (6,6)
pos[5] = (3,8)
pos[6] = (2.75,8.5)
pos[7] = (3,4.5)
pos[8] = (4,4)
pos[9] = (3.5,3)
pos[10] = (3,2.5)
pos[11] = (2.5,2)
pos[12] = (7,7)
pos[13] = (7.5,8)
pos[14] = (8,7)
pos[15] = (6.5,5)
pos[16] = (7.5,4)
pos[17] = (8,3.5)
pos[18] = (2.5,7.5)
pos[19] = (2.5,9)
pos[20] = (3,9)
pos[21] = (2,5)
pos[22] = (2,4)
pos[23] = (3.5,4.25)
pos[24] = (4,2)
pos[25] = (2.5,3.25)
pos[26] = (1.5,1)
pos[27] = (2.75,1)
pos[28] = (7.25,8.5)
pos[29] = (8,8.5)
pos[30] = (8.25,7.25)
pos[31] = (8.25,6.75)
pos[32] = (7,5.5)
pos[33] = (7.5,3)
pos[34] = (8.5,4.25)
pos[35] = (8.5,3)

In [ ]:
def print_graph(node_labels=False, edge_labels=False, save=False):
    fig = plt.figure(figsize = (10, 10))   # an in-line figure
    plt.gca().set_axis_off()

    # nodes
    nx.draw_networkx_nodes(bronchial_tree,pos,nodelist=non_terminal_nodes, node_size=250, node_color="white")
    nx.draw_networkx_nodes(bronchial_tree,pos,nodelist=terminal_nodes + [origin], node_size=250, node_color="red")

    # edges
    nx.draw_networkx_edges(bronchial_tree,pos)

    # node labels
    if node_labels:
        labels = dict()
        for n in terminal_nodes + [origin]:
            labels[n] = bronchial_tree.node[n][COUNT]
        nx.draw_networkx_labels(bronchial_tree,pos,labels,font_family='sans-serif')

    # edge labels
#     if edge_labels:
#         nx.draw_networkx_edge_labels(bronchial_tree, pos, edge_labels = edge_labels)

    plt.axis('off')
    plt.title('Lung')
    plt.show()
    if save:
        fig.savefig("bronchial_tree.png") # save as png

In [ ]:
def set_up():

    bronchial_tree.node[initial_infected_node][COUNT] = initial_load
    for n in range(1, len(bronchial_tree.node)):
        bronchial_tree.node[n][COUNT] = 0

    assert 1.0 >= expulsion_rate >= 0.0, "Expulsion rate must be between 0.0 and 1.0"

In [ ]:
# STAHLER ORDERS

order = 1.0

queued_nodes = []
for child_node, parent_node in bronchial_tree.edges(terminal_nodes):
    bronchial_tree[child_node][parent_node]['stahler_order'] = order
    queued_nodes.append(parent_node)

# Order the queued list (highest value first)
queued_nodes = sorted(set(queued_nodes), reverse=True)

while len(queued_nodes) > 0:
    node = queued_nodes.pop(0)
    edges = bronchial_tree.edges(node, data=True)
    
    # Check that all child edges have a horsfield order (i.e. only parent is unfilled)
    child_orders = [e[2]['stahler_order'] for e in edges if 'stahler_order' in e[2].keys()]
    assert len(child_orders) == len(bronchial_tree.edges(node)) - 1
    
    if max(child_orders) ==  min(child_orders):
        new_order = max(child_orders) + 1
    else:
        new_order = max(child_orders)
    
    parent_node = [n for n in bronchial_tree.neighbors(node) if n < node][0]
    
    # Set the new order on the aprent edge
    bronchial_tree[node][parent_node]['stahler_order'] = new_order
    # Add the parent node to the queue
    if parent_node != origin:
        queued_nodes.append(parent_node)
    queued_nodes = sorted(set(queued_nodes), reverse=True)
    

In [ ]:
# HORSFIELD & CUMMING
order = 1.0

queued_nodes = []
for child_node, parent_node in bronchial_tree.edges(terminal_nodes):
    bronchial_tree[child_node][parent_node]['horsfield_order'] = order
    queued_nodes.append(parent_node)

# Order the queued list (highest value first)
queued_nodes = sorted(set(queued_nodes), reverse=True)

while len(queued_nodes) > 0:
    node = queued_nodes.pop(0)
    edges = bronchial_tree.edges(node, data=True)
    
    # Check that all child edges have a horsfield order (i.e. only parent is unfilled)
    child_orders = [e[2]['horsfield_order'] for e in edges if 'horsfield_order' in e[2].keys()]
    assert len(child_orders) == len(bronchial_tree.edges(node)) - 1
    new_order = max(child_orders) + 1
    
    parent_node = [n for n in bronchial_tree.neighbors(node) if n < node][0]
    
    # Set the new order on the aprent edge
    bronchial_tree[node][parent_node]['horsfield_order'] = new_order
    # Add the parent node to the queue
    if parent_node != origin:
        queued_nodes.append(parent_node)
    queued_nodes = sorted(set(queued_nodes), reverse=True)

In [ ]:
def inhale():
    bacteria_locations = dict()
    final_bacteria_locations = dict()
    for a in terminal_nodes:
        final_bacteria_locations[a] = 0
    
    bacteria_locations[origin] = bronchial_tree.node[origin][COUNT]
    adjacent_nodes = bronchial_tree.neighbors(origin)
    
    while len(bacteria_locations) > 0:
        node, bacteria_count = bacteria_locations.popitem()
        neighbours = [neighbour for neighbour in bronchial_tree.neighbors(node) if neighbour > node]
        new_locs = split(node, neighbours, bacteria_count)
        for a in new_locs:
            if a in terminal_nodes:
                final_bacteria_locations[a] = new_locs[a]
            else:
                bacteria_locations[a] = new_locs[a]
    
    bronchial_tree.node[0][COUNT] = 0
    for l in final_bacteria_locations:
        bronchial_tree.node[l][COUNT] += final_bacteria_locations[l]

In [ ]:
def split(node, neighbours, bacteria_count):
    new_locs = dict()
    
    if len(neighbours) == 1:
        new_locs[neighbours[0]] = bacteria_count
    else:
        if split_method == 'even':
            split_value = split_even(len(neighbours), bacteria_count)
        elif split_method == 'stahler':
            split_value = split_stahler(node, neighbours, bacteria_count)
        elif split_method == 'horsfield':
            split_value = split_horsfield(node, neighbours, bacteria_count)
        else:
            raise Exception, "Invalid split method"
        new_locs[neighbours[0]] = split_value[0]
        new_locs[neighbours[1]] = split_value[1]
    
    return new_locs

In [ ]:
def split_even(num_of_neighbours, bacteria_count):
    # TODO - cater for >2
    assert num_of_neighbours == 2
    if bacteria_count%2 == 0:
        split_value = [bacteria_count/2,bacteria_count/2]
    else:
        choice = np.random.randint(0,2)
        if choice:
            split_value = [bacteria_count/2,(bacteria_count/2) + 1]
        else:
            split_value = [(bacteria_count/2) + 1,bacteria_count/2]
    return split_value

In [ ]:
def split_random(num_of_neighbours, bacteria_count):
    # TODO - cater for >2
    assert num_of_neighbours == 2
    
    split_1 = np.random.randint(0,bacteria_count+1)
    split_2 = bacteria_count - split_1
    
    return [split_1, split_2]

In [ ]:
def split_stahler(node, neighbours, count):
    # TODO - cater for > 2
    edge1 = bronchial_tree[node][neighbours[0]]
    edge2 = bronchial_tree[node][neighbours[1]]
    
    total_stahler = edge1['stahler_order'] + edge2['stahler_order']
    
    left_branch_count = round(count * (edge1['stahler_order']/total_stahler))
    right_branch_count = count - left_branch_count
    
    return [left_branch_count, right_branch_count]

In [ ]:
def split_horsfield(node, neighbours, count):
    # TODO - cater for > 2
    edge1 = bronchial_tree[node][neighbours[0]]
    edge2 = bronchial_tree[node][neighbours[1]]
    
    total_horsfield = edge1['horsfield_order'] + edge2['horsfield_order']
    
    left_branch_count = round(count * (edge1['horsfield_order']/total_horsfield))
    right_branch_count = count - left_branch_count
    
    return [left_branch_count, right_branch_count]

In [ ]:
def growth_death():
    for tn in terminal_nodes:
        count = bronchial_tree.node[tn][COUNT]
        bronchial_tree.node[tn][COUNT] = int(round(count + (count * growth_rate) - (count * death_rate)))

In [ ]:
def exhale():
    # TODO - are all exhaled?
    expelled = 0
    for tn in terminal_nodes:
        expelled_from_node = int(round(bronchial_tree.node[tn][COUNT] * expulsion_rate))
        expelled += expelled_from_node
        bronchial_tree.node[tn][COUNT] -= expelled_from_node

In [ ]:
def cough():
    pass
    

In [ ]:
# Create data structures
data = []

def update_data():
    data_at_timestep = dict()
    for n in terminal_nodes:
        data_at_timestep[n] = bronchial_tree.node[n][COUNT]
    data.append(data_at_timestep)

In [ ]:
set_up()
update_data()

for t in range(1,time_limit+1):
    inhale()
    growth_death()
    exhale()
    update_data()
    # print_graph()
print_graph()

In [ ]:
max_y = 0
for n in terminal_nodes:
    data_node = []
    for row in data:
        data_node.append(row[n])
    plt.plot(data_node, label=n)
    max_y = max(max_y, max(data_node))
plt.axis([0,time_limit,0,max_y+10])
plt.title("Bronchopulmonary segments")
plt.legend(loc=0)
plt.show()

In [ ]:
max_y = 0
for lobe in lobe_nodes:
    data_lobe = []
    for row in data:
        count = 0
        for node in lobe_nodes[lobe]:
            count += row[node]
        data_lobe.append(count)
    plt.plot(data_lobe, label=lobe)
    max_y = max(max_y, max(data_lobe))
plt.axis([0,time_limit,0,max_y+10])
plt.title("Lobes")
plt.legend(loc=0)
plt.show()

In [ ]:
max_y = 0
for lobe in lobe_nodes:
    data_lobe = []
    for row in data:
        count = []
        for node in lobe_nodes[lobe]:
            count.append(row[node])
        data_lobe.append((sum(count) * 1.0)/len(count))
    plt.plot(data_lobe, label=lobe)
    max_y = max(max_y, max(data_lobe))
plt.axis([0,time_limit,0,max_y+10])
plt.title("Lobes (average per segment)")
plt.legend(loc=4)
plt.show()

In [ ]:
max_y = 0
for lung in lung_nodes:
    data_lung = []
    for row in data:
        count = 0
        for node in lung_nodes[lung]:
            count += row[node]
        data_lung.append(count)
    plt.plot(data_lung, label=lung)
    max_y = max(max_y, max(data_lung))
plt.axis([0,time_limit,0,max_y+10])
plt.title("Lungs")
plt.legend(loc=4)
plt.show()

In [ ]:
max_y = 0
for lung in lung_nodes:
    data_lung = []
    for row in data:
        counts = []
        for node in lung_nodes[lung]:
            counts.append(row[node])
        data_lung.append((sum(counts) * 1.0)/len(counts))
    plt.plot(data_lung, label=lung)
    max_y = max(max_y, max(data_lung))
plt.axis([0,time_limit,0,max_y+10])
plt.title("Lungs (average per segment)")
plt.legend(loc=4)
plt.show()

In [ ]:

print ordered_list